In [ ]:
import os
import json
import shutil
import pandas as pd
from quantification_functions import postprocess_LSA_seg, get_all_LSA_metrics, get_ROI_location_from_mask, crop_TOF_and_seg

In [ ]:
############# Input #############
ID="001"    # ID number
root_dir = "/path/to/LUMEN/Quantification"
data_dir = os.path.join("/path/to/data/folder", ID)
original_LSA_PA_mask_path = os.path.join(data_dir, f"{ID}_upsampled_TOF_LSA_PA_mask.nii.gz")
TOF_path = os.path.join(data_dir, f"{ID}_upsampled_TOF.nii.gz") # ensure that this is the isotropically upsampled TOF
seg_path = os.path.join(data_dir, f"{ID}_seg.nii.gz")           # segmentation mask from DS6
slicer_path = "/path/to/slicer.exe"  # TODO: replace with path to Slicer on your device  # on Mac it might be "/Applications/Slicer.app/Contents/MacOS/Slicer"; on Windows it might be something like "C:\"Slicer 5.6.1"\Slicer.exe"
#################################
print(f"Using segmentation at {seg_path}")
mip_volume_property_path = os.path.join(root_dir, "MIP_VolumeProperty.vp")
LSA_PA_mask_path = os.path.join(data_dir, f"{ID}_final_LSA_PA_mask.nii.gz")
LSA_ROI_dict_save_path = os.path.join(data_dir, f"{ID}_LSA_ROI_location.json")

### 1. Postprocess LSA PA mask and get ROI location (automatic)

Smooth the LSA perfusion area (PA) mask -- this is recommended if the mask was originally defined on a lower resolution template and coregistered to the high-resolution upsampled TOF image. Smoothing can remove stair artefacts in the mask. We utilised the joint smoothing method from Slicer due to its superior performance. The command below will launch Slicer, save smoothed mask to the `LSA_PA_mask_path` specified above, and close Slicer automatically.

In [ ]:
# Tip: if {slicer_path} cannot be found, try to replace it with the full path of Slicer here
!{slicer_path} --no-splash --python-script smooth_LSA_PA_mask.py -mask {original_LSA_PA_mask_path} -volume {TOF_path} -output {LSA_PA_mask_path}

Locate two LSA ROIs from the smoothed mask and save the ROI location to a json file. If detecting more than two islands (may occur due to artefact from smoothing), keep the two largest ones.

In [ ]:
LSA_ROI_dict = get_ROI_location_from_mask(LSA_PA_mask_path, original_LSA_PA_mask_path)

with open(LSA_ROI_dict_save_path, 'w') as json_file:
    json.dump(LSA_ROI_dict, json_file, indent=4)

### 2. Select side and crop ROI (automatic)

In [ ]:
side = "Left"  #"Left" or "Right"
save_dir = os.path.join(data_dir, side)
LSA_ROI_dict = json.load(open(LSA_ROI_dict_save_path, "r"))
cropped_TOF_path, cropped_seg_path, cropped_mask_path = crop_TOF_and_seg(ID, 
                                                                    side, 
                                                                    save_dir, 
                                                                    TOF_path, 
                                                                    seg_path,
                                                                    LSA_ROI_dict=LSA_ROI_dict, 
                                                                    LSA_PA_mask_path=LSA_PA_mask_path,
                                                                    remove_top_surface_islands=False)

### 3. Manual correction and keep LSAs (manual)

- Launch Slicer. Load the cropped TOF image, `labelled_seg.nii.gz` and `helper_endpoints.json` into Slicer. 

- Go to `Segment Editor` module and correct the segmentation of the LSAs. Remove all non-LSA islands.

- Save corrected segmentation to default filename in the same folder, `labelled_seg.nii.gz.nii.seg.nrrd`.

### 4. Postprocessing and endpoint detection (automatic)
Run the code below to postprocess the segmentation for LSAs. It will create a new folder named `Postprocessed_LSA_seg`, in which there will be the postprocessed segmentation mask with the LSA branches originating from different stems labelled separately, as well as json files storing the detected endpoints for each cluster of branches. 

In [ ]:
corrected_nrrd_path = os.path.join(save_dir, "labelled_seg.nii.gz.nii.seg.nrrd")
postprocessed_save_dir = os.path.join(save_dir, "Postprocessed_Separate_LSAs")
postprocessed_LSA_seg_filename = "postprocessed_LSA_seg.nii.gz"

# Remove previous results if they exist
if os.path.exists(postprocessed_save_dir):
    shutil.rmtree(postprocessed_save_dir)

postprocessed_LSA_seg_path = postprocess_LSA_seg(corrected_nrrd_path, ref_img_path=cropped_seg_path, 
                                                save_dir=postprocessed_save_dir, 
                                                out_seg_name=postprocessed_LSA_seg_filename, 
                                                ROI_mask_path=cropped_mask_path)

Check if the endpoints and segments look correct. If modifying the endpoints, save the corrected endpoint lists to the same json files.

### 5. Centerline extraction and construct LSA trees (automatic)
Run the code below to automatically launch Slicer, extract the centrelines of the LSA segmentation, and save them to a new `Results` folder. If the results look right, simply close the window or stop this cell.

P.S.  This step may run for a few minutes depending on your machine. If you are re-running this step and already has a `Results` folder, this will over-write the result files!

In [ ]:
results_dir = os.path.join(save_dir, "Results")
if os.path.exists(results_dir):
    shutil.rmtree(results_dir)

!{slicer_path} --no-splash --python-script extract_centerline_in_slicer.py -load_dir {postprocessed_save_dir} -seg_filename {postprocessed_LSA_seg_filename} -TOF_path {cropped_TOF_path} -results_dir {results_dir} -mip_volume_property {mip_volume_property_path}

### 6. Compute LSA metrics

In [ ]:
metrics = ['length', 'tortuosity', 'mean_curvature', 'max_curvature']
summary_results, full_results = get_all_LSA_metrics(results_dir, metrics)

df = pd.DataFrame([summary_results]).T
df.columns = ['Value']
display(df)